In [6]:
import argparse
import os
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')



import keras
import tensorflow

## Data Setting

In [7]:
test = pd.read_csv('test.csv')
test = test[['Process','Temp', 'Current', 'NG']]
test.head()

,Process,Temp,Current,NG
0,6,75.041983,1.552,0
1,6,75.312474,1.492,0
2,6,76.533664,1.699,0
3,6,73.740424,1.705,0
4,6,77.045185,1.733,0


# Threshold (required for inspection)

In [8]:
with open('./threshold_list.pkl', 'rb') as f:
    threshold_list = pickle.load(f)

# Test set to sequence data

In [9]:
def create_sequences(X, time_steps=3):
    Xs = []
    for i in range(len(X)-time_steps+1):
        Xs.append(X.iloc[i:(i+time_steps)].values)
    return np.array(Xs)

# Inference

In [10]:
def Inference(data, idx):
    process = data['Process'].unique().item()
    
    model = keras.models.load_model('./model/LSTMAE_'+str(process)+'.h5') # model load
    with open('./scaler/Scaler_'+str(process)+'.pkl', 'rb') as f:  # scaler load
        scaler = pickle.load(f)
        
    
    test = scaler.transform(data[diagnosis])
    
    test = pd.DataFrame(test, columns=diagnosis)
        
    X_test = create_sequences(test)
    
    pred = model.predict(X_test)
    mse = np.mean(np.power(X_test-pred, 2), axis=1)
    mean_mse = []
    for i in range(len(mse)):
        mean_mse.append(mse[i].mean())
        
    thr = threshold_list[process-1]
    
    result = pd.DataFrame()
    label = []
    label_sum = 0

    index_list = [(str(i)+'-'+str(i+2)) for i in range(idx-1)]
    for j in mean_mse:
        if j>thr:
            label.append('abnormal')
            label_sum+=1
        else:
            label.append('normal')
            
    
    result['Label'] = label
    result['index'] = index_list
    
    result.set_index('index', inplace=True)
            
    return model, result, label_sum

# Process Inspection

In [11]:
Log = pd.DataFrame()
diagnosis = ['Temp', 'Current'] 
for i in range(len(test)):
    print('In process proceeding...')
    event = test[i:i+1] 
    Log = Log.append(event)  
    time.sleep(4)
    display(Log)  
    if len(Log) >= 3:
        model, diagnosis_result, label_sum = Inference(Log, i)
        model
        if label_sum >= 1:
            print('Anomaly detected!!')
            print('------------------End Process------------------\n\n')
            print('***Where an Anomaly Detected ?***')
            display(diagnosis_result)
            break

In process proceeding...


,Process,Temp,Current,NG
0,6,75.041983,1.552,0


In process proceeding...


,Process,Temp,Current,NG
0,6,75.041983,1.552,0
1,6,75.312474,1.492,0


In process proceeding...


...

,Process,Temp,Current,NG
0,6,75.041983,1.552,0
1,6,75.312474,1.492,0
2,6,76.533664,1.699,0
3,6,73.740424,1.705,0
4,6,77.045185,1.733,0
5,6,76.472491,1.670,0
6,6,71.041983,1.745,0


1/1 [==============================] - 1s 620ms/step
In process proceeding...


,Process,Temp,Current,NG
0,6,75.041983,1.55200,0
1,6,75.312474,1.49200,0
2,6,76.533664,1.69900,0
3,6,73.740424,1.70500,0
4,6,77.045185,1.73300,0
5,6,76.472491,1.67000,0
6,6,71.041983,1.74500,0
7,6,108.162219,1.12196,1


1/1 [==============================] - 1s 596ms/step
Anomaly detected!!
------------------End Process------------------


***Where an Anomaly Detected ?***


,Label
index,
0-2,normal
1-3,normal
2-4,normal
3-5,normal
4-6,normal
5-7,abnormal
